In [3]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 6 - from bag to matrix
</center>
</h1>
<div class=h1_cell>
<p>
This week I want to head in a slightly different direction than the bag of words approach. I want you to consider (and program) a co-occurrence matrix (comat). The comat can lead toward some very cool NLP methods.
</div>

<h2>
Here's the idea
</h2>
<div class=h1_cell>
<p>
With bag of words, we threw away context information. If I look up bag_of_words['fawn'], I can see how many times fawn appeared but I don't know anything about what context it appeared in. The context I will be interested to start with are the two words on either side of fawn, i.e., one word on left and one word on right. If fawn is at beginning or end of sentence, then we will only count one word.
<p>
To get this new information about context, I will have to go through all the sentences again, and for each word I will update what word is on either side of it. Let me try a small example. Pretend we only have 3 sentences.
<p>
<pre>
I love programming. I love Math. I tolerate Biology.
</pre>
<p>
Check out the comat I would build from these sentences.
<p>
<img src='https://cdn-images-1.medium.com/max/1600/1*1p0geczj9KbJvwYi25B2Jg.png'>
<p>
You should notice that the row names and column names are symmetric. They are made up of the unique words in all the sentences. You should also note that sentences matter. For instance, if you remove the periods, then you would get `Math` coming right before `I`. So should be an entry for `[Math, I]` of `1`. But it is zero. We do not cross sentence boundaries to check for co-occurrences.
<p>
Also note that we are looking at either side of a word for co-occurrences. Hence, `I` shows up with `love` twice and `love` shows up with `I` twice.
<p>
Finally note that the above matrix is counting a period as a word. I think that is interesting, using punctuation instead of throwing it out, but I did not do it. To match my results, use your sentence_wrangler, which throws out punctuation among other things.
</div>

<h2>
There is one parameter
</h2>
<div class=h1_cell>
<p>
The parameter for a comat is how far from the target word we check. In the example above, the parameter value is 1 - check 1 word on either side. But I could set it to a larger integer. If I make it big enough, I'll likely get most of the sentence as the context.
<p>
Reminder: parameters like this in data science are often called hyper-parameters. The "K" in KNN is a hyper-parameter.
<p>
I won't ask you to parameterize your algorithm for the main assignment: you can assume it is a constant 1. If anyone needs make-up points for past homework, I'll give you extra credit points if you can parameterize your algorithm to use a parameter. So look 2 words out or 3 words out, etc. The larger the parameter, the more context you are picking up.
</div>

<h2>
Doing things a little differently
</h2>
<div class=h1_cell>
<p>
Normally I would guide you through main steps and have you fill in pieces. I am going to cut you loose for this assignment. You can solve the problem in any way you want as long as you match my results.
<p>
What you will need to get started:
<ul>
<li>gothic_table
<li>sentence_wrangler plus what it needs as params.
<li>cosine_similarity
</ul>
<p>
I hope you have inferred from the sample comat above that your comat will be N by N where N = 24834. Yikes. Every unique word is a row/column label. This may take a fairly big chunk of your laptop's virtual memory. Here is a piece of code that you might find useful. It tells you what percentage of memory you have remaining.
<pre>
<code>
import psutil
print(psutil.virtual_memory())  # percent is what is remaining
</code>
</pre>
<p>
I was ok on my laptop with 16GB memory. If you run into trouble on your machine, see me.
<p>
I also took time to compute various things. Did I mention we are now working on a `24834x24834 matix`? Things take time. On my slow machine, worst I got was 9 minutes. You might look at the magic-command `%time`: sometimes easier to use than setting up start and end time, e.g., `%time some_function()` will give you the elapsed time for getting a return value.
</div>

<h2>
Challenge 1
</h2>
<div class=h1_cell>
<p>
Build the comat using sentences from gothic_table with a spread of 1, i.e., 1 word on either side of target word. Please use sorted labels on your comat. This is different than the sample comat above which uses the order the words are seen. So your matrix should have these as the first 10 rows and columns.
<p>
<pre>
'aaem'
'ab'
'aback'
'abaft'
'abandon
'abandoned'
'abandoning'
'abandonment'
'abaout'
'abased'
</pre>
<p>
Use any raw Python data structures you like. I am also ok with a pandas dataframe if you want.
<p>
I've given you my results to match against below.
</div>

In [4]:
import pandas as pd

gothic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQqRwyE0ceZREKqhuaOw8uQguTG6Alr5kocggvAnczrWaimXE8ncR--GC0o_PyVDlb-R6Z60v-XaWm9/pub?output=csv',
                          encoding='utf-8')

In [5]:
gothic_table.head()


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
len(gothic_table)

19579

<h2>
Go to it
</h2>
<div class=h1_cell>
<p>
Add your own code cells to create and test your code.
</div>

In [7]:
from nltk.corpus import stopwords
swords = stopwords.words('english')

from nltk.tokenize import WordPunctTokenizer
word_punct_tokenizer = WordPunctTokenizer()

import string
punctuation = string.punctuation

#legals = 'abcdefghijklmnopqrstuvwxyz'

def sentence_wrangler(sentence, swords, punctuation):
    tokes = word_punct_tokenizer.tokenize(sentence)
    result = []
    removed = []
    check = 0
    for word in tokes:
        if word.lower() not in swords:
            if word in punctuation:
                removed.append(word)
            else:
                for i in word:
                    if i in punctuation:
                        check = 0
                    else:
                        check = 1
                if check == 1:
                    result.append(word.lower())
                else:
                    removed.append(word.lower())
        else:
            removed.append(word.lower())
    return (result, removed)

def cosine_similarity(v1,v2):
    AB = 0.0
    A = 0.0
    B = 0.0
    for i in range(len(v1)):
        AB += (v1[i] * v2[i])
        A += (v1[i]**2)
        B += (v2[i]**2)  
    return AB/((A*B)**(0.5))

In [8]:
#here is a starting cell - add new ones with + tab above.
def comat_maker(k, table, stop_words, punctuation):
    all_words = {}
    manifest = {}
    for i in range(len(table)):
        new = {}
        vec = table.loc[i, 'text']
        sentence = sentence_wrangler(vec, stop_words, punctuation)[0]
        for j in range(len(sentence)):
            if sentence[j] not in all_words:
                manifest[sentence[j]] = 0
                if j < len(sentence)-k and j == 0:
                    new = {}
                    for z in range(k):
                        new[sentence[j+(z+1)]] = 1
                    all_words[sentence[j]] = new
                elif j < len(sentence)-k:
                    new = {}
                    for z in range(k):
                        new[sentence[j-(z+1)]] = 1
                    for z in range(k):
                        new[sentence[j+(z+1)]] = 1
                    all_words[sentence[j]] = new
                elif j == len(sentence)-k:
                    new = {}
                    for z in range(k):
                        new[sentence[j-(z+1)]] = 1
                    all_words[sentence[j]] = new
            else:
                if j < len(sentence)-k and j == 0:
                    for z in range(k):
                        if sentence[j+(z+1)] in all_words[sentence[j]]:
                            all_words[sentence[j]][sentence[j+(z+1)]] += 1
                        else:
                            all_words[sentence[j]][sentence[j+(z+1)]] = 1
                elif j < len(sentence)-k:
                    for z in range(k):
                        if sentence[j-(z+1)] in all_words[sentence[j]]:
                            all_words[sentence[j]][sentence[j-(z+1)]] += 1
                        else:
                            all_words[sentence[j]][sentence[j-(z+1)]] = 1     
                        if sentence[j+(z+1)] in all_words[sentence[j]]:
                            all_words[sentence[j]][sentence[j+(z+1)]] += 1
                        else:
                            all_words[sentence[j]][sentence[j+(z+1)]] = 1
                elif j == len(sentence)-k:
                    for z in range(k):
                        
                        if sentence[j-(z+1)] in all_words[sentence[j]]:
                            all_words[sentence[j]][sentence[j-(z+1)]] += 1
                        else:
                            all_words[sentence[j]][sentence[j-(z+1)]] = 1 
                        
    return (all_words, manifest)

<h2>
Match against my results
</h2>
<div class=h1_cell>
<p>
I don't know how you are implementing the comat so I'll just describe the results I got below. Show me that you get the same.

<ul>
<li>The count of 0s in row 'monster' is 24765.
<li>The cosine_similarity between the rows 'frankenstein' and 'monster' is 0.07273929674533079
<li>The sum of the 'frankenstein' row is 32.
<li>The sum of the 'monster' row is 74.
</ul>
</div>

In [24]:
#show me your match
info = comat_maker(1, gothic_table, swords, punctuation)

print (len(info[1]) - len(info[0]['monster']))
print len(info[0]['monster'])

def build_vectors(dict1, dict2):
    d1 = []
    d2 = []
    #match dictionary size
    for word in sorted(dict1):
        if word not in dict2:
            dict2[word] = 0
    for word in sorted(dict2):
        if word not in dict1:
            dict1[word] = 0
    #create vectors out of values
    for word in sorted(dict1):
        d1.append(dict1[word])
    for word in sorted(dict2):
        d2.append(dict2[word])
    return (d1, d2)

vecs = build_vectors(info[0]['monster'], info[0]['frankenstein'])

frank_monster = cosine_similarity(vecs[0], vecs[1])
print frank_monster

total_frank = 0
for count in info[0]['frankenstein'].values():
    total_frank += count
print total_frank

total_monster = 0
for count in info[0]['monster'].values():
    total_monster += count
print total_monster


24875
69
0.0727392967453
32
74


<h2>
Challenge 2
</h2>
<div class=h1_cell>
<p>
Do the first part of KNN. Find all the distances from a specific word as a row in comat. Use cosine_similarity. Match my results.
<p>
BTW: this is where things slowed down and I was getting 9 minutes on my slow machine.
</div>

In [19]:
test1 = [u'uncomplainingly', u'hernani', u'skulking',u'mimes', u'decipherer', u'ever', u'thought', u'sentience', u'never', u'indistinctly']
test2 = [u'unrelenting', u'tempora', u'madam', u'fie', u'moses', u'victor', u'cousin', u'margaret', u'demoniac', u'fallacious']
def word_distances(matrix, word):
    all_distances = []
    for other_word in test1:
        vecs = build_vectors(matrix[0][word], matrix[0][other_word])
        distance = cosine_similarity(vecs[0], vecs[1])
        all_distances.append((other_word, distance))
    sorted_distances = sorted(all_distances, key=lambda pair: pair[1], reverse=True)
    return sorted_distances
    
matrix = comat_maker(1, gothic_table, swords, punctuation)

In [20]:
%time monster_distances = word_distances(matrix, 'monster')

CPU times: user 25.1 ms, sys: 4.05 ms, total: 29.2 ms
Wall time: 26.1 ms


In [21]:
monster_distances[:10]

[(u'uncomplainingly', 0.23145502494313785),
 (u'hernani', 0.2182178902359924),
 (u'skulking', 0.2182178902359924),
 (u'mimes', 0.2182178902359924),
 (u'decipherer', 0.2182178902359924),
 (u'ever', 0.19495617958510608),
 (u'thought', 0.19014352249619199),
 (u'sentience', 0.17251638983558856),
 (u'never', 0.1697162329953409),
 (u'indistinctly', 0.1690308509457033)]

In [15]:
%time frank_distances = word_distances(matrix, 'frankenstein')

CPU times: user 2.57 ms, sys: 621 µs, total: 3.19 ms
Wall time: 2.64 ms


In [16]:
frank_distances[:10]

[(u'unrelenting', 0.3333333333333333),
 (u'tempora', 0.3333333333333333),
 (u'madam', 0.31277162108561213),
 (u'fie', 0.2721655269759087),
 (u'moses', 0.2721655269759087),
 (u'victor', 0.26653757937535255),
 (u'cousin', 0.26470046277199466),
 (u'margaret', 0.2545875386086578),
 (u'demoniac', 0.23570226039551587),
 (u'fallacious', 0.23570226039551587)]

<h2>
Closing notes
</h2>
<div class=h1_cell>
<p>
If we had more time in the quarter, I'd like to follow up on the next step we can take once we have the comax. The general idea is that the comax is a very sparse matrix, consisting of mostly 0 entries. And as you have seen, it takes time to search it. There are linear-algebra  techniques for reducing a sparse matrix into a more dense matrix and hence, making them more computationally tractable to work with. One I particular wish we had time to explore is Singular-value Decomposition (or SVD): https://en.wikipedia.org/wiki/Singular-value_decomposition. This is used with the Glove algorithm to take a comax, exactly like the one you built, and reduce it: https://nlp.stanford.edu/projects/glove/.
<p>
An alternative approach is one called word-to-vec (or word2vec). This uses a shallow neural-net to reduce each word to a vector of "features". Here is a good overview: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/. You may have seen some of the cool things they can do with these feature vectors, e.g., take the vector for 'king', subtract the vector for 'man' and add the vector for 'woman. The resulting vector is 'queen'.
<p>
If I was going to teach a follow-on course to this one, I would definitely want to follow the neural-net and deep-learning path further. If anyone is interested in doing a reading (for credit) on these topics in summer or next year, see me.
</div>